 # Description
 We are aiming to complete the analysis of the data that has been made available to us. This step adds even more information to our initial set. This time we will check, among other things, whether flight delays depend on the route or weather conditions.

  However, before we get to that, you need to configure the notebook properly, just like in the previous steps.

In [1]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sqlalchemy import create_engine
from psycopg2 import connect

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


 ## Connection to the database

In [2]:
username = 'postgres'
password = 'password'

host = 'localhost'
database = 'airlines'
port = 5432

In [3]:
url = f"postgresql://{username}:{password}@{host}:{port}/{database}"

engine = create_engine(url)

In [4]:
def read_sql_table(table_name):
    file = pd.read_sql("SELECT * FROM" +" " + table_name +';'"", engine)
    return file

In [5]:
filename = "../data/processed/flight_df_02.csv"
flight_df = pd.read_csv(
        filename,
        sep=';',
        decimal='.',
        encoding='UTF-8',
    )

flight_df

,Unnamed: 0.1,Unnamed: 0,id,month,day_of_month,day_of_week,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,...,weather_delay,nas_delay,security_delay,late_aircraft_delay,year,is_delayed,is_weekend,distance_agg,manufacture_year,manufacture_year_agg
0,0,0,1,1,1,2,9E,N931XJ,3290,10874,...,NaN,NaN,NaN,NaN,2019,False,0,"(500, 600]",2008.0,"(2005, 2008]"
1,1,1,2,1,1,2,OH,N723PS,5495,10874,...,NaN,NaN,NaN,NaN,2019,True,0,"(300, 400]",2004.0,"(2002, 2005]"
2,2,2,3,1,1,2,OH,N525EA,5416,10874,...,NaN,NaN,NaN,NaN,2019,False,0,"(300, 400]",NaN,NaN
3,3,3,4,1,1,2,OH,N706PS,5426,10874,...,NaN,NaN,NaN,NaN,2019,False,0,"(300, 400]",2004.0,"(2002, 2005]"
4,4,4,5,1,1,2,OH,N262PS,5440,10874,...,NaN,NaN,NaN,NaN,2019,False,0,"(300, 400]",2004.0,"(2002, 2005]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6922919,6922919,7422040,7422032,12,30,1,MQ,N818AE,3744,12511,...,0.0,0.0,0.0,31.0,2019,True,0,"(400, 500]",2002.0,"(1999, 2002]"
6922920,6922920,7422041,7422033,12,30,1,MQ,N821AE,4235,12511,...,NaN,NaN,NaN,NaN,2019,False,0,"(300, 400]",2002.0,"(1999, 2002]"
6922921,6922921,7422043,7422035,12,31,2,MQ,N245NN,4040,12511,...,129.0,0.0,0.0,0.0,2019,True,0,"(300, 400]",2016.0,"(2014, 2017]"
6922922,6922922,7422044,7422036,12,31,2,MQ,N806AE,3744,12511,...,0.0,0.0,0.0,71.0,2019,True,0,"(400, 500]",2001.0,"(1999, 2002]"


 # Enriching with  `airport_list`

In [6]:
airport_list_df_first = read_sql_table('airport_list')

airport_list_df = airport_list_df_first.copy()
airport_list_df

,id,origin_airport_id,display_airport_name,origin_city_name,name
0,1,11638,Fresno Air Terminal,"Fresno, CA","FRESNO YOSEMITE INTERNATIONAL, CA US"
1,2,13342,General Mitchell Field,"Milwaukee, WI","MILWAUKEE MITCHELL AIRPORT, WI US"
2,3,13244,Memphis International,"Memphis, TN","MEMPHIS INTERNATIONAL AIRPORT, TN US"
3,4,15096,Syracuse Hancock International,"Syracuse, NY","SYRACUSE HANCOCK INTERNATIONAL AIRPORT, NY US"
4,5,10397,Atlanta Municipal,"Atlanta, GA",ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...
...,...,...,...,...,...
92,93,13198,Kansas City International,"Kansas City, MO","KANSAS CITY INTERNATIONAL AIRPORT, MO US"
93,94,10423,Austin - Bergstrom International,"Austin, TX","AUSTIN BERGSTROM INTERNATIONAL AIRPORT, TX US"
94,95,15370,Tulsa International,"Tulsa, OK","OKLAHOMA CITY WILL ROGERS WORLD AIRPORT, OK US"
95,96,13303,Miami International,"Miami, FL","MIAMI INTERNATIONAL AIRPORT, FL US"


In [7]:
test = airport_list_df.drop_duplicates(subset='origin_airport_id').shape[0]
test == airport_list_df.shape[0]

True

In [8]:
flight_df = pd.merge(
    left=flight_df,
    right=airport_list_df[['origin_airport_id', 'origin_city_name']],
    how='left',
    on=['origin_airport_id']
)
flight_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6922924 entries, 0 to 6922923
Data columns (total 36 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Unnamed: 0.1          int64  
 1   Unnamed: 0            int64  
 2   id                    int64  
 3   month                 int64  
 4   day_of_month          int64  
 5   day_of_week           int64  
 6   op_unique_carrier     object 
 7   tail_num              object 
 8   op_carrier_fl_num     int64  
 9   origin_airport_id     int64  
 10  dest_airport_id       int64  
 11  crs_dep_time          int64  
 12  dep_time              float64
 13  dep_delay             float64
 14  dep_time_blk          object 
 15  crs_arr_time          int64  
 16  arr_time              float64
 17  arr_delay_new         float64
 18  arr_time_blk          object 
 19  cancelled             float64
 20  crs_elapsed_time      float64
 21  actual_elapsed_time   float64
 22  distance              float64
 23  distanc

In [9]:
tmp_airport_list_df = airport_list_df.rename(
    columns = {'origin_city_name': 'destination_city_name'})

flight_df = pd.merge(
    left=flight_df,
    right=tmp_airport_list_df[['origin_airport_id', 'destination_city_name']],
    how='left',
    left_on=['dest_airport_id'],
    right_on=['origin_airport_id']
)

flight_df = (flight_df
            .drop(columns='origin_airport_id_y')
            .rename(columns={'origin_airport_id_x': 'origin_airport_id'})
            )

flight_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6922924 entries, 0 to 6922923
Data columns (total 37 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Unnamed: 0.1           int64  
 1   Unnamed: 0             int64  
 2   id                     int64  
 3   month                  int64  
 4   day_of_month           int64  
 5   day_of_week            int64  
 6   op_unique_carrier      object 
 7   tail_num               object 
 8   op_carrier_fl_num      int64  
 9   origin_airport_id      int64  
 10  dest_airport_id        int64  
 11  crs_dep_time           int64  
 12  dep_time               float64
 13  dep_delay              float64
 14  dep_time_blk           object 
 15  crs_arr_time           int64  
 16  arr_time               float64
 17  arr_delay_new          float64
 18  arr_time_blk           object 
 19  cancelled              float64
 20  crs_elapsed_time       float64
 21  actual_elapsed_time    float64
 22  distance          

 ## Analysis by airports and routes

In [10]:
top_airports_origin_df = (flight_df
                          .groupby(by=['origin_airport_id']).size()
                         )
top_airports_origin_df

origin_airport_id
10135     4938
10136     2234
10140    23403
10141      732
10146     1001
         ...  
15897      264
15919    13839
15991      701
16218     1911
16869      204
Length: 359, dtype: int64

In [11]:
top_airports_destination_df = (flight_df
                          .groupby(by=['dest_airport_id']).size()
                         )
top_airports_destination_df

dest_airport_id
10135     4943
10136     2233
10140    23437
10141      732
10146     1001
         ...  
15897      264
15919    13866
15991      713
16218     1907
16869      207
Length: 359, dtype: int64

 # Enrichment with weather data

In [12]:
airport_weather_df_first = read_sql_table('airport_weather')

airport_weather_df = airport_weather_df_first.copy()
airport_weather_df

,id,station,name,date,awnd,pgtm,prcp,snow,snwd,tavg,...,wt09,wesd,wt10,psun,tsun,sn32,sx32,tobs,wt11,wt18
0,1,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-01,4.70,NaN,0.14,0.0,0.0,64.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1,2,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-02,4.92,NaN,0.57,0.0,0.0,56.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2,3,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-03,5.37,NaN,0.15,0.0,0.0,52.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,4,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-04,12.08,NaN,1.44,0.0,0.0,56.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,5,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-05,13.42,NaN,0.00,0.0,0.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46221,46222,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",2020-03-27,3.58,146.0,0.21,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
46222,46223,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",2020-03-28,6.93,1535.0,1.29,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
46223,46224,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",2020-03-29,16.55,1408.0,0.02,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
46224,46225,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",2020-03-30,13.42,817.0,0.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [13]:
airport_weather_df = airport_weather_df[['station', 'name', 'date',
                                         'prcp', 'snow', 'snwd', 'tmax',
                                         'awnd']]

In [14]:
airport_weather_df = pd.merge(
    left=airport_weather_df,
    right=airport_list_df[['origin_airport_id', 'name']],
    how='inner',
    on='name'
)

In [49]:
airport_weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43394 entries, 0 to 43393
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   station            43394 non-null  object 
 1   name               43394 non-null  object 
 2   date               43394 non-null  object 
 3   prcp               43368 non-null  float64
 4   snow               29578 non-null  float64
 5   snwd               29007 non-null  float64
 6   tmax               43386 non-null  float64
 7   awnd               43386 non-null  float64
 8   origin_airport_id  43394 non-null  int64  
dtypes: float64(5), int64(1), object(3)
memory usage: 3.3+ MB


 ## Join `airport_weather_df` and `flight_df`

In [15]:
airport_weather_df['date'] = pd.to_datetime(airport_weather_df['date'])

In [16]:
flight_df['date'] = (flight_df['year'].astype(str)
                     + '-' + flight_df['month'].astype(str)
                     + '-' + flight_df['day_of_month'].astype(str)
                    )
flight_df['date'] = pd.to_datetime(flight_df['date'])

In [17]:
flight_df=pd.merge(
    left=flight_df,
    right=airport_weather_df,
    how='left',
    on=['date','origin_airport_id']
)
flight_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6922924 entries, 0 to 6922923
Data columns (total 45 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   Unnamed: 0.1           int64         
 1   Unnamed: 0             int64         
 2   id                     int64         
 3   month                  int64         
 4   day_of_month           int64         
 5   day_of_week            int64         
 6   op_unique_carrier      object        
 7   tail_num               object        
 8   op_carrier_fl_num      int64         
 9   origin_airport_id      int64         
 10  dest_airport_id        int64         
 11  crs_dep_time           int64         
 12  dep_time               float64       
 13  dep_delay              float64       
 14  dep_time_blk           object        
 15  crs_arr_time           int64         
 16  arr_time               float64       
 17  arr_delay_new          float64       
 18  arr_time_blk          

In [19]:
flight_df.to_csv("../data/processed/flight_df_03.csv",
                sep=';')